In [37]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.analysis.make_analysis_data as ad
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def user_period_data(df, user_id, period):
    return (
        df.set_index("user_id", drop=False)
        .loc[user_id]
        .set_index("date", drop=False)
        .loc[period]
        .reset_index(drop=True)
    )

Dev

In [ ]:
df = hd.read_sample('XX7')
hd.inspect(df)

## Improve txn data production 

In [11]:
dfr = hd.read_raw_sample("777")
hd.inspect(dfr)

Time for read_raw_sample: 3.68 seconds
(682,656, 27)


,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,688293,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,"9572 24jan12 , tcs bowdon , bowdon gb - pos",Debit,25.030001,No Tag,No Tag,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-24,U
1,688291,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,warrington b.c. - d/d,Debit,26.000000,Gym Membership,Gym Membership,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-23,U


In [99]:
clean = md.clean_data(dfr)

Time for rename_cols                   : 0.03 seconds
Time for clean_headers                 : 0.00 seconds
Time for drop_unneeded_vars            : 0.03 seconds
Time for add_year_month_variable       : 0.05 seconds
Time for drop_first_and_last_month     : 0.06 seconds
Time for lowercase_categories          : 0.96 seconds
Time for drop_missing_txn_desc         : 0.05 seconds
Time for gender_to_female              : 0.03 seconds
Time for credit_debit_to_debit         : 0.01 seconds
Time for sign_amount                   : 0.00 seconds
Time for missings_to_nan               : 0.01 seconds
Time for zero_balances_to_missing      : 0.00 seconds
Time for add_tag                       : 0.33 seconds
Time for tag_corrections               : 0.56 seconds
Time for add_tag_group                 : 0.12 seconds
Time for drop_type1_dups               : 0.30 seconds
Time for order_and_sort                : 0.15 seconds
Time for clean_data                    : 2.75 seconds


In [102]:
created = md.create_vars(clean)

Time for balances                      : 2.83 seconds
Time for income                        : 0.33 seconds
Time for age                           : 0.06 seconds
Time for entropy_spend_tag_counts      : 0.30 seconds
Time for region_name                   : 0.70 seconds
Time for create_vars                   : 4.24 seconds


In [111]:
selected = md.select_sample(created)

Time for select_sample                 : 0.42 seconds


In [112]:
validated = md.validate_data(selected)

All validation checks passed.
Time for validate_data                 : 0.10 seconds


dft = hd.read_sample("777")
hd.inspect(dft)

In [10]:
dft[dft.tag_group.eq("spend") & ~dft.debit]

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,debit,female,latest_balance,merchant_business_line,postcode,tag_auto,tag_manual,tag_up,ym,yob,balance,income,age,entropy_sptac,region
5,2994248,2012-11-01,14777,-0.95,interest (net),NaN,spend,NaN,2013-01-07,2013-01-07,247061,2015-01-13 05:19:00,lloyds bank,current,False,0.0,729.590027,account provider,sw11 5,interest charges,NaN,interest income,201211,1986.0,-479.021179,14587.920410,26.0,2.636914,London
75,2994191,2012-12-03,14777,-0.64,interest (net),NaN,spend,NaN,2013-01-07,2013-01-07,247061,2015-01-13 05:19:00,lloyds bank,current,False,0.0,729.590027,account provider,sw11 5,interest charges,NaN,interest income,201212,1986.0,-816.681213,14587.920410,26.0,2.631123,London
143,2994150,2013-01-02,14777,-0.83,interest (net),NaN,spend,NaN,2013-01-07,2013-01-07,247061,2015-01-13 05:19:00,lloyds bank,current,False,0.0,729.590027,account provider,sw11 5,interest charges,NaN,interest income,201301,1986.0,616.748901,16765.099609,27.0,2.474526,London
157,3099875,2013-01-11,14777,-26.00,www.asos.com cd 7024,asos,spend,NaN,2013-01-07,2013-01-07,247060,2015-01-13 05:19:00,lloyds bank,current,False,0.0,-733.090027,asos,sw11 5,clothes - everyday or work,NaN,NaN,201301,1986.0,1051.259521,16765.099609,27.0,2.474526,London
201,3473093,2013-02-01,14777,-0.79,interest (net),NaN,spend,NaN,2013-01-07,2013-01-07,247061,2015-01-13 05:19:00,lloyds bank,current,False,0.0,729.590027,account provider,sw11 5,interest charges,NaN,interest income,201302,1986.0,-331.930908,16765.099609,27.0,2.660896,London
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42347,730104126,2020-03-09,564777,-2.91,asda groceries onl,asda,spend,NaN,2020-01-18,2020-01-18,1564118,2020-04-17 00:10:00,lloyds bank,current,False,1.0,171.960007,asda supermarket,cf83 1,refunded purchase,NaN,refunded purchase,202003,1986.0,-2477.620117,13367.200195,34.0,2.463656,Wales
42365,731489613,2020-03-11,564777,-70.75,david lloyd leisur,david lloyd,spend,NaN,2020-01-18,2020-01-18,1564118,2020-04-17 00:10:00,lloyds bank,current,False,1.0,171.960007,david lloyd,cf83 1,gym membership,NaN,gym membership,202003,1986.0,-2287.800293,13367.200195,34.0,2.463656,Wales
42384,733366375,2020-03-16,564777,-2.00,asda groceries onl,asda,spend,NaN,2020-01-18,2020-01-18,1564118,2020-04-17 00:10:00,lloyds bank,current,False,1.0,171.960007,asda supermarket,cf83 1,refunded purchase,NaN,refunded purchase,202003,1986.0,-2264.270020,13367.200195,34.0,2.463656,Wales
42431,749353485,2020-03-23,564777,-14.11,asda groceries onl,asda,spend,NaN,2020-01-18,2020-01-18,1564118,2020-04-17 00:10:00,lloyds bank,current,False,1.0,171.960007,asda supermarket,cf83 1,refunded purchase,NaN,refunded purchase,202003,1986.0,-2593.070068,13367.200195,34.0,2.463656,Wales


In [6]:
vl.spend_tag(dft)

{'communication',
 'finance',
 'hobbies',
 'household',
 'motor',
 nan,
 'other_spend',
 'retail',
 'services',
 'travel'}

In [7]:
vl.spend_tag(dft)

{'communication',
 'finance',
 'hobbies',
 'household',
 'motor',
 'other_spend',
 'retail',
 'services',
 'travel'}

## Test analysis data

In [36]:
df = hd.read_analysis_data()
hd.inspect(df)

Time for read_analysis_data: 0.78 seconds
(20,035, 32)


txn_count_sa  txn_count_ca   balance_ca  balance_sa  \
user_id date                                                              
107     2014-04-30           0.0           0.0 -1140.444336       700.0   
        2014-05-31           0.0          93.0 -1140.444336       700.0   

                    sa_inflows  sa_outflows  sa_net_inflows  \
user_id date                                                  
107     2014-04-30         0.0          0.0             0.0   
        2014-05-31         0.0          0.0             0.0   

                    sa_scaled_inflows  sa_scaled_outflows  \
user_id date                                                
107     2014-04-30                0.0                 0.0   
        2014-05-31                0.0                 0.0   

                    sa_scaled_net_inflows  log_monthly_spend  \
user_id date                                                   
107     2014-04-30                    0.0           6.078765   
        2014-05-31                    0.0           7.205212   

                    tag_spend_services  tag_spend_household  tag_spend_travel  \
user_id date                                                                    
107     2014-04-30            0.386698             0.435039          0.020161   
        2014-05-31            0.103035             0.474202          0.075385   

                    tag_spend_motor  tag_spend_other_transfers  \
user_id date                                                     
107     2014-04-30         0.158102                        0.0   
        2014-05-31         0.165883                        0.0   

                    tag_spend_finance  tag_spend_other_spend  \
user_id date                                                   
107     2014-04-30           0.000000               0.000000   
        2014-05-31           0.043537               0.059416   

                    tag_spend_retail  tag_spend_communication  \
user_id date                                                    
107     2014-04-30          0.000000                 0.000000   
        2014-05-31          0.031372                 0.047169   

                    tag_spend_earnings  tag_spend_benefits  tag_spend_savings  \
user_id date                                                                    
107     2014-04-30                 0.0                 0.0                0.0   
        2014-05-31                 0.0                 0.0                0.0   

                    tag_spend_hobbies  tag_spend_other_income  \
user_id date                                                    
107     2014-04-30                0.0                     0.0   
        2014-05-31                0.0                     0.0   

                    tag_spend_pensions        income  log_income  female  \
user_id date                                                               
107     2014-04-30                 0.0  27530.926758   10.223065     0.0   
        2014-05-31                 0.0  27530.926758   10.223065     0.0   

                     age      region  entropy_sptac  
user_id date                                         
107     2014-04-30  30.0  South West       2.282854  
        2014-05-31  30.0  South West       2.373440

In [37]:
df.isna().sum()

txn_count_sa                   0
txn_count_ca                   0
balance_ca                     0
balance_sa                     0
sa_inflows                     0
sa_outflows                    0
sa_net_inflows                 0
sa_scaled_inflows              0
sa_scaled_outflows             0
sa_scaled_net_inflows          0
log_monthly_spend              0
tag_spend_services             0
tag_spend_household            0
tag_spend_travel               0
tag_spend_motor                0
tag_spend_other_transfers      0
tag_spend_finance              0
tag_spend_other_spend          0
tag_spend_retail               0
tag_spend_communication        0
tag_spend_earnings             0
tag_spend_benefits             0
tag_spend_savings              0
tag_spend_hobbies              0
tag_spend_other_income         0
tag_spend_pensions             0
income                         0
log_income                     0
female                         0
age                            0
region    

## pipeline dev

In [5]:
fs.ls("3di-data-mdb/raw/")

['3di-data-mdb/raw/',
 '3di-data-mdb/raw/20200630_UserLoginsForNeedham.csv',
 '3di-data-mdb/raw/mdb_000.csv',
 '3di-data-mdb/raw/mdb_000.parquet',
 '3di-data-mdb/raw/mdb_777.csv',
 '3di-data-mdb/raw/mdb_777.parquet',
 '3di-data-mdb/raw/mdb_X77.csv',
 '3di-data-mdb/raw/mdb_X77.parquet',
 '3di-data-mdb/raw/mdb_XX7.csv',
 '3di-data-mdb/raw/mdb_XX7.parquet',
 '3di-data-mdb/raw/mdb_costa.csv']

In [4]:
ha.read_csv("s3://3di-data-mdb/raw/mdb_000.csv", nrows=3, sep="|")

,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,686979264,564777,2020-01-18,1986.0,NaN,CF83 1,W01001427,W02000313,F,2018-10-15,1564118,Lloyds Bank,Current,171.96,nespresso uk ltd,Debit,15.60,"Food, Groceries, Household",No Tag,"Food, Groceries, Household",Nespresso,Nespresso,2020-01-18,2020-04-17 00:10:00,2020-01-19,1900-01-01,C
1,335712120,299777,2015-05-10,1970.0,NaN,BS11 0,E01014498,E02003014,M,2018-04-05,427663,Santander,Current,0.00,"direct debit payment to paypal payment ref 5ae22226hzlca, mandate no 0016",Debit,34.99,Enjoyment,No Tag,Enjoyment,PayPal,PayPal,2015-05-10,2018-09-19 19:58:00,2018-04-07,1900-01-01,C
2,108299228,313777,2015-07-21,1996.0,NaN,KT22 8,E01030553,E02006365,F,2015-12-01,454866,HSBC,Current,75.96,co-op group xx0273 guildford,Debit,13.22,Groceries,No Tag,"Food, Groceries, Household",Co-op,Co-op Supermarket,2015-07-26,2019-08-03 18:22:00,2015-12-02,2018-09-03,U
